# Линейная регрессия: t-тест

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_params, summary_col # вывод результатов тестирования

from scipy.stats import t # t-распределение и критические значения

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

## t-тест: значимость коэффициентов

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Тестируем нулевую гипотезу $H_0:\beta=0$

Альтернативная гипотеза $H_1:\beta\ne 0$

Тестовая статистика $$t=\frac{\hat{\beta}}{s.e.(\beta)}$$

Критическое значение $t_{cr}=t_{df}(\alpha)$, где  $df=n-k-1$ --- степени свободы для t-распределения

$\color{purple}{\text{\textbf{Значим ли коэффициент} }\beta ?}$

**<span style="color:purple">Способ 1:</span>** используем $t-value$

* Отвергаем гипотезу $H_0$ при $|t|>t_{cr}$, <i><b><span style="color: purple">коэффициент значим</span></b><br>
* Не отвергаем гипотезу $H_0$ при $|t|<t_{cr}$, <i><b><span style="color: purple">коэффициент незначим</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем $P-value$

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">коэффициент значим</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">коэффициент незначим</span> </b><br>

---

<i><b><span style="color: purple">Сравниваем самостоятельно или автоматизированно</span> <br>

<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span></b>

---

</div>

Для набора данных `sleep75` рассмотрим линейную регрессию

$$
    sleep = \beta_0+\beta_1 totwrk+\beta_2 age+\beta_3 south+\beta_4 male+\beta_5 smsa+\beta_6 yngkid+\beta_7 marr+\beta_8 union+\beta_9 \log(hrwage)+error
$$

In [ ]:
# импорт данных
df = pd.read_csv('sleep75.csv')

## Спецификация и подгонка

In [ ]:
# спецификация модели через формулу
mod = ols(formula='sleep~totwrk+age+south+male+smsa+yngkid+marr+union+np.log(hrwage)', data=df)
# подгонка модели
res = mod.fit()

In [ ]:
# Общий отчет
res.summary(slim=True)

## Гипотеза

Тестируем значимость коэффициента $\beta_{age}$, т.е. гипотезу $H_0: \beta_{age} = 0$

## Уровень значимости
Пусть уровень значимости $\alpha=5\%=0.05$

Число наблюдений $n$, по которым была оценена модель

In [ ]:
# Число наблюдений, по которым была оценена модель
res.nobs

Число регрессоров $k$, по которым была оценена модель

In [ ]:
# Число регрессоров, по которым была оценена модель
res.df_model

Степени свободы для t-распределения $df=n-k-1$

In [ ]:
# Степени свободы для t-распределения
res.df_resid

### Составляющие t-теста для коэффициентов

In [ ]:
# стандартные ошибки s.e.(beta) для каждого коэффициента с округлением
res.bse.round(3)

In [ ]:
# тестовые t-статистики для каждого коэффициента с округлением
res.tvalues.round(3)

In [ ]:
# P-значения для t-статистик с округлением
res.pvalues.round(4)

In [ ]:
# Составляющие t-теста для коэффициента(ов) в одной таблице
summary_params(res).round(3)

## Критическое значение t-распределения

In [ ]:
# уровень значимости
sign_level = 0.05
# критическое значение t-распределения
t_cr = t.isf(sign_level/2, df=res.df_resid).round(3)
t_cr

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `age`

## Какие коэффиценты значимы?

## Значимость всех коэффициентов

- проверяем значимость коэффициентов используя P-value (сравниваем самостоятельно или автоматизированно)

- проверяем значимость коэффициентов используя t_cr (сравниваем самостоятельно или автоматизированно)

**<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span>**

In [ ]:
# проверим значимость коэффициентов используя P-value
df_sign = summary_params(res, alpha=sign_level).round(3)
df_sign['significance'] = df_sign.apply(lambda x: 'Значим' if x['P>|t|']<sign_level else 'Незначим', axis=1)
df_sign

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

In [ ]:
# проверим значимость коэффициентов используя t_cr
df_sign = summary_params(res, alpha=sign_level).round(3)
df_sign['significance'] = df_sign.apply(lambda x: 'Значим' if np.abs(x['t'])>t_cr else 'Незначим', axis=1)
df_sign

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

## Значимость выбранных коэффициентов
Потестируем значимость коэффициентов $\beta_{totwrk}$ и $\beta_{male}$ на уровне значимости 5%

In [ ]:
res.t_test('totwrk=0, male=0')

## Критическое значение t-распределения

In [ ]:
# уровень значимости
sign_level = 0.05
# 5%-критическое значение t-распределения
t_cr = t.isf(q=sign_level/2, df=res.df_resid).round(3)
t_cr

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 5% значим коэффициент при `totwrk`

---

# Общий t-тест

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Тестируем $H_0:\beta=\theta$ ($\theta$ - заданное число)

Тестовая статистика $$t=\frac{\hat{\beta}-\theta}{s.e.(\beta)}$$

Критическое значение $t_{cr}=t_{df}(\alpha)$, где  $df=n-k-1$ --- степени свободы для t-распределения

$\color{purple}{\text{\bf Значимо ли коэффициент }\beta \text{ \bf отличается от числа } \theta?}$

**<span style="color: purple">Способ 1:</span>** используем $t-value$

* Отвергаем гипотезу $H_0$ при $|t|>t_{cr}$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf значимо отличается от числа } \theta}$<br> 
* Не отвергаем гипотезу $H_0$ при $|t|<t_{cr}$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf незначимо отличается от числа } \theta}$<br> 

**<span style="color: purple">Способ 2:</span>** используем $P-value$

* Отвергаем гипотезу $H_0$  при $P<\alpha$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf значимо отличается от числа } \theta}$<br> 
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, $\color{purple}{\text{\bf коэффициент }\beta \text{ \bf незначимо отличается от числа } \theta}$<br> 

---

<i><b><span style="color: purple">Сравниваем самостоятельно</span> <br>

<span style="color:#59afe1">ОБЯЗАТЕЛЬНО ПЕЧАТАЕМ ВЫВОД</span></b>

---

</div>

Для набора данных `Labour` рассмотрим регрессию 

$$
\log(output) = \beta_0+\beta_1\log(capital) + \beta_2\log(labour)+error
$$

In [ ]:
# импорт данных
df = pd.read_csv('Labour.csv')

## Спецификация и подгонка

In [ ]:
# спецификация модели через формулу
mod = ols(formula='np.log(output)~np.log(capital)+np.log(labour)', data=df)
# подгонка модели
res = mod.fit()

In [ ]:
# Общий отчет
res.summary(slim=True)

## Гипотеза
Тестируется гипотеза $H_0:\beta_{capital}=0.5$ 


## Уровень значимости
Пусть уровень значимости $\alpha=1\%=0.01$

## Результаты тестирования

In [ ]:
# Составляющие общего t-теста для коэффициента(ов) в одной таблице
res.t_test('np.log(capital)=0.5')

## Критическое значение t-распределения

In [ ]:
# зададим уровень значимости
sign_level = 0.01
# критическое значение t-распределения
t_cr = t.ppf(q=1-sign_level/2, df=res.df_resid).round(3)
t_cr

**<span style="color:#59afe1">ВЫВОД</span>**: На уровне значимости 1% коэффициент при `log(capital)` значимо отличается от числа 0.5